#### Задача 1 (10 баллов). 

Напишем мини-стратегическую игру, в которой игроку предлагается управлять городом (SimCity видели когда-нибудь?) У нас есть собственно класс "город", внутри города есть постройки разных типов, которые мы, как градоправитель, строим. Также у города есть атрибуты: уровень счастья жителей, уровень экологии, образования, миграции и так далее (на что вашей фантазии хватит). Постройки к этим атрибутам добавляют и прибавляют баллы. Можно завести тип постройки, которая на каждом ходе игры приносит в городской бюджет деньги (налоговая служба?.. заводы?). Постройки необходимо реализовать с помощью наследования, у некоторых построек могут быть свои методы, например, больница может устраивать медосмотры и вакцинации и поднимать этим уровень благосостояния жителей, а на заводе с рандомной вероятностью может случиться авария, которая понизит экологию. Не забудьте реализовать головной класс "игра", в котором будет метод play, можно в нем хранить и городской бюджет. 

Фантазия приветствуется: чем оригинальнее будет ваша игра, тем выше шанс, что я поставлю дополнительный балл. 

In [1]:
from time import sleep
import random

In [19]:
# Это будет город переселенцев на Марс

class Settlement:
    def __init__(self, name):
        self.name = name
        self.happiness = 10
        self.ecology = 10
        self.money = 15000 # бюджет на старте
        self.education = 5
        self.health = 1
        self.inhabitants = 3
        self.lots = []

    def buildl(self, lot):
        if self.money >= lot.cost:
            self.lots.append(lot)
            self.money -= lot.cost
            lot.affsetstats(self)
            print(f'\nВы построили {lot.name}\nБюджет поселения - {self.money}')

    def collectmoney(self):
        rev = 0
        for lot in self.lots:
            if isinstance(lot, Motherlode):
                rev += lot.plusrevenue()
        self.money += rev
        print(f'Пополнение бюджета поселения на {rev}, всего в бюджете {self.money}')                    

    def stats(self):
        print(f'\nПоселение: {self.name}')
        print(f'Жители: {self.inhabitants}')
        print(f'Бюджет поселения: {self.money}')
        print(f'Уровень счастья жителей: {self.happiness}')
        print(f'Уровень экологии: {self.ecology}')
        print(f'Уровень образования: {self.education}')
        print(f'Уровень здоровья: {self.health}')
        

    def showlots(self):
        if self.lots:
            print(f'\nВ поселении сейчас есть')
            for lot in self.lots:
                print(f'- {lot.name}')

class Lot:
    def __init__(self, name, cost, happiness=0, ecology=0, education=0):
        self.name = name
        self.cost = cost
        self.happiness = happiness
        self.ecology = ecology
        self.education = education

    def affsetstats(self, settlement): # affect settlement stats
        settlement.happiness += self.happiness
        settlement.ecology += self.ecology
        settlement.education += self.education    
    

class ResidentialPod(Lot):  # жилое помещение
    def __init__(self):
        super().__init__(name='Residential Pod', cost=1000, happiness=2, ecology=0)
    
    def addpeople(self, settlement):
        settlement.inhabitants += 5
        print('Население увеличилось!')

class ResearchStation(Lot): # исследовательский центр
    def __init__(self):
        super().__init__(name='Research Station', cost=5000, education=5, happiness=2, ecology=-2)

class MedicalUnit(Lot): # медучреждение
    def __init__(self):
        super().__init__(name='Medical Unit', cost=6000, happiness=3, ecology=0)

    def vaccination(self, settlement):
        settlement.happiness += 1
        settlement.health += 5
        print('Медицинский центр провёл вакцинацию. Уровень здоровья повысился!')    
        
    def chekup(self, settlement):
        settlement.happiness += 1
        settlement.health += 5
        print('Медицинский центр провёл плановое обследование населения. Уровень здоровья повысился!')

class Farm(Lot):  # ферма
    def __init__(self):
        super().__init__(name='Farm', cost=2000, happiness=1, ecology=-1)

class Kindergarten(Lot): # дет сад
    def __init__(self):
        super().__init__(name='Kindergarten', cost=3000, education=3, happiness=2)

class School(Lot): # школа для детей
    def __init__(self):
        super().__init__(name='School', cost=5000, education=5, happiness=2)


class PowerPlant(Lot): # электростанция
    def __init__(self):
        super().__init__(name='Power Plant', cost=5000, happiness=1, ecology=-3)
    
    def accident(self, settlement):
        if random.random() < 0.15:  # 15%
            settlement.happiness -= 4
            settlement.ecology -= 10
            print('ЧП на электростанции! Снизился уровень экологии и счастья жителей.')

class LifesupportSystem(Lot): # система жизнеобеспечения для помещений (как на мкс)
    def __init__(self):
        super().__init__(name='LS System', cost=4000, happiness=5, ecology=5)

class Motherlode(Lot): # класс для money участков
    def __init__(self, name, cost, revperturn, happiness=0, ecology=0):
        super().__init__(name, cost, happiness, ecology)
        self.revperturn = revperturn
    
    def plusrevenue(self):
        return self.revperturn

class Supermarket(Motherlode): # магазин (+ деньги)
    def __init__(self):
        super().__init__(name='Supermarket', cost=3000, revperturn=2000, happiness=2)

class Taxplace(Motherlode): # налоги
    def __init__(self):
        super().__init__(name='Tax Org', cost=5000, revperturn=1000, happiness=-3)

    def subsidy(self, settlement):
        if random.random() < 0.3: # 30%
            settlement.money += 3000 # субсидии с Земли...
            print(f'Братья земляне решили послать вам деньги на развитие поселения.')


class AndroidFactory(Lot): # do androids dream...
    def __init__(self):
        super().__init__(name='android factory', cost=8000, happiness=1, ecology=-5)
    
    def accident(self, settlement):
        if random.random() < 0.1:  # 10%
            settlement.happiness -= 5
            settlement.ecology -= 10
            print('Авария на заводе! Снизился уровень экологии и счастья жителей.')

class RecyclingCentre(Lot): # reuse reduce recycle
    def __init__(self):
        super().__init__(name='Recycling Centre', cost=3000, happiness=3, ecology=10)

class CommunityCentre(Lot): # типа дома культуры
    def __init__(self):
        super().__init__(name='Community Centre', cost=5000, education=1, happiness=5)


class Game:

    def __init__(self, settlname):
        self.settlement = Settlement(settlname)

    def playoneturn(self):
        print(f'\n*** Новый ход ***')
        # сначала собираем деньги
        self.settlement.collectmoney()

        # рандомные чп + деятельность других зданий
        for lot in self.settlement.lots:
            if isinstance(lot, AndroidFactory) or isinstance(lot, PowerPlant):
                lot.accident(self.settlement)
            elif isinstance(lot, ResidentialPod):
                lot.addpeople(self.settlement)
            elif isinstance(lot, MedicalUnit):
                lot.chekup(self.settlement)
                lot.vaccination(self.settlement)
            elif isinstance(lot, Taxplace):
                lot.subsidy(self.settlement)
                        

        self.settlement.stats()
        self.settlement.showlots()

        self.lotchoice()

    def lotchoice(self):
        print('\n*** Хотите построить новый участок? ***')
        print('1. Tax Org (cost: 5000)')
        print('2. Supermarket (cost: 3000)')
        print('3. Residential Pod (cost: 1000)')
        print('4. Life support system (cost: 4000)')
        print('5. Farm (cost: 2000)')
        print('6. Research Station (cost: 5000)')
        print('7. Kindergarten (cost: 3000)')
        print('8. School (cost: 5000)')
        print('9. Medical unit  (cost: 6000)')
        print('10. Community Centre (cost: 5000)')
        print('11. Power Plant (cost: 5000)')
        print('12. Android factory (cost: 8000)')
        print('13. Recycling centre (cost: 3000)')
        print('14. Ничего не строить')

        choice = input('Выберите участок (1-14):')
        
        if choice == '1':
            self.build('tax')
        elif choice == '2':
            self.build('market')
        elif choice == '3':
            self.build('pod')
        elif choice == '4':
            self.build('ls')
        elif choice == '5':
            self.build('farm')
        elif choice == '6':
            self.build('res')
        elif choice == '7':
            self.build('kind')
        elif choice == '8':
            self.build('school')
        elif choice == '9':
            self.build('medical')
        elif choice == '10':
            self.build('comm')
        elif choice == '11':
            self.build('pow')                            
        elif choice == '12':
            self.build('android')
        elif choice == '13':
            self.build('rrr')    
        elif choice == '14':
            print('Вы решили ничего не строить')
        else:
            print('Попробуйте ещё. Выберите 1-14')
            self.lotchoice()

    def build(self, lottype):
        if lottype == 'tax':
            self.settlement.buildl(Taxplace())
        elif lottype == 'market':
            self.settlement.buildl(Supermarket())
        elif lottype == 'pod':
            self.settlement.buildl(ResidentialPod())
        elif lottype == 'ls':
            self.settlement.buildl(LifesupportSystem())
        elif lottype == 'farm':
            self.settlement.buildl(Farm())
        elif lottype == 'res':
            self.settlement.buildl(ResearchStation())
        elif lottype == 'kind':
            self.settlement.buildl(Kindergarten())
        elif lottype == 'school':
            self.settlement.buildl(School())
        elif lottype == 'medical':
            self.settlement.buildl(MedicalUnit())
        elif lottype == 'comm':
            self.settlement.buildl(CommunityCentre())
        elif lottype == 'pow':
            self.settlement.buildl(PowerPlant())
        elif lottype == 'android':
            self.settlement.buildl(AndroidFactory())
        elif lottype == 'rrr':
            self.settlement.buildl(RecyclingCentre())        



In [20]:
if __name__ == '__main__':
    game = Game('Mars3000')

    for _ in range(15):
        game.playoneturn()


*** Новый ход ***
Пополнение бюджета поселения на 0, всего в бюджете 15000

Поселение: Mars3000
Жители: 3
Бюджет поселения: 15000
Уровень счастья жителей: 10
Уровень экологии: 10
Уровень образования: 5
Уровень здоровья: 1

*** Хотите построить новый участок? ***
1. Tax Org (cost: 5000)
2. Supermarket (cost: 3000)
3. Residential Pod (cost: 1000)
4. Life support system (cost: 4000)
5. Farm (cost: 2000)
6. Research Station (cost: 5000)
7. Kindergarten (cost: 3000)
8. School (cost: 5000)
9. Medical unit  (cost: 6000)
10. Community Centre (cost: 5000)
11. Power Plant (cost: 5000)
12. Android factory (cost: 8000)
13. Recycling centre (cost: 3000)
14. Ничего не строить

Вы построили Tax Org
Бюджет поселения - 10000

*** Новый ход ***
Пополнение бюджета поселения на 1000, всего в бюджете 11000

Поселение: Mars3000
Жители: 3
Бюджет поселения: 11000
Уровень счастья жителей: 7
Уровень экологии: 10
Уровень образования: 5
Уровень здоровья: 1

В поселении сейчас есть
- Tax Org

*** Хотите построить